In [28]:
import numpy as np 
import os
import re
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from sklearn.model_selection import train_test_split

# Preparação dos arquivos 

## Tendo como uma das tarefas eliminar as imagens que não possuem roda, criei o código abaixo. 
## - Defino o caminho do diretorio de anotações e de imagens. 
## - Utilizo a biblioteca os, para navegar até o diretorio, pegar o nome de todos os arquivos e ordenalos em ordem crescente. 
## - Percorro cada um dos arquivos, abro ele com a função open da biblioteca os, verifico se em seu conteudo existe uma linha que começa com a str "0", se sim, salvo o nome do arquivo na lista nomes_ants_com_rodas.
## - Por fim, percorro cada posição da lista de anotações, altero o .txt para .jpg, e salvo na lista nomes_imgs_com_rodas.

In [2]:
dir_anotacoes = 'data-20230312T190552Z-001/data/annotations'
dir_imagens = 'data-20230312T190552Z-001/data/images/'

arquivos = sorted(os.listdir(dir_anotacoes))

nomes_ants_com_rodas = []
nomes_imgs_com_rodas = []

for nome_arquivo in arquivos:

    with open(os.path.join(dir_anotacoes, nome_arquivo), 'r') as arquivo:
        conteudo = arquivo.read()
        padrao = r'\b0(?![\d.])'
        
        if re.search(padrao, conteudo):
            nomes_ants_com_rodas.append(nome_arquivo)

for nome in nomes_ants_com_rodas:
    nome_img = nome.replace('.txt', '.jpg')
    nomes_imgs_com_rodas.append(nome_img)

## - Percorro a lista de anotações e salvo as bbox posição 1,2,3,4 em uma lista, criando uma lista de listas chamada bboxes.
## - Altero o tipo de cada posição para float.
## - Por fim crio o y como um array numpy.

In [3]:
bboxes = []

for nome_arquivo in nomes_ants_com_rodas:

    with open(os.path.join(dir_anotacoes, nome_arquivo), 'r') as arquivo:

        for linha in arquivo:
            
            if re.match(r'^0', linha):
                dados = linha.strip().split()
                bboxes.append(dados[1:])

for bbox in bboxes:
    for posicao in range(len(bbox)):
        bbox[posicao] = float(bbox[posicao])

y = np.array(bboxes)

## - Percorro o diretorio de imagens, abrindo cada uma com o cv2, aletrando para preto e branco e normalizando os dados das imagens para 0,1, salvando as informações em uma lista de imagens.

## - Percorro a lista de imagens, alterando o tamanho de cada imagem pela metade, salvo a informação em uma lista de mesmo tamanho. 

## - Foi necessário redmencionar, pois meus testes apresentaram melhor resultado ao usar mais imagens, em vez de imagens de maior tamanho, e com o intuito de balancear a rede conforme a capacidade de meu computador, optei por reduzir as imagens a metade de seu tamanho original. 

## - Crio o x como uma array numpy

In [4]:
imagens = [cv2.imread(dir_imagens + nome, 0) / 255 for nome in nomes_imgs_com_rodas]

imagens = [cv2.resize(img, (320, 160)) for img in imagens]

x = np.array(imagens)

## - Crio uma função para conseguir plotar um retangulo com as informações YOLO disponiveis no bbox, ela recebe a quantidade de imagens que quero mostrar, o as imagens(x) e os bbox(y), defino que a imagem a ser mostrada é que esta em x na posição i, e que meus pontos para o retangulo são os que estão em y na posição i. Faço o calculo para ter os 4 pontos e atribuo cada um deles em uma variavel, que já esta ajustada com a altura e largura correta. Crio minha img com o cv2 e a função rectangle, que ira mostrar a imagem, o retangulo definido pelos 4 pontos, em co preta(0), e com linha de espessura 4.

In [5]:
def show_imagens(quantidade, x, y):
    
    l, a, _ = x[0].shape
    for i in range(quantidade):
        imagem = x[i]
        x_norm, y_norm, w_norm, h_norm = y[i]
        x_ = int((x_norm - w_norm / 2) * a)
        y_ = int((y_norm + h_norm / 2) * l)
        w_ = int((x_norm + w_norm/2) * a)
        h_ = int((y_norm - h_norm/2) * l)
        img = cv2.rectangle(imagem, (x_, y_), (w_, h_), 0, 4)
        cv2.imshow(f'imagem{i}', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

## - Defino meu x e y de treino e teste, pegando 20% da amostra para teste.

In [6]:
x_train, x_test, y_train , y_test = train_test_split(x, y, test_size=0.2)

## Apresento o shape, como exigencia da rede, adiciono uma nova camada ao array numpay e apresento novamente o shape ajustado.  

In [7]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(576, 160, 320)
(576, 4)
(144, 160, 320)
(144, 4)


In [16]:
xTrain = x_train.shape
xTest = x_test.shape


In [17]:
x_train = np.reshape(x_train, (xTrain[0],xTrain[1],xTrain[2], 1))
x_test = np.reshape(x_test, (xTest[0],xTest[1], xTest[2], 1))

In [18]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(576, 160, 320, 1)
(576, 4)
(144, 160, 320, 1)
(144, 4)


## - Apresento imagens que serão utilizadas para treinar, defini como 10, mas alterando o número, irá mostrar a quantidade que você definir. 

In [19]:
show_imagens(10, x_train, y_train)

## - Fiz varios testes, com mais e com menos kernels e neuronios, 32, 16 e 64 se mostraram eficientes e com consumo adequado de memoria e processamento. 
## - Crio uma rede sequencial, com duas camadas de convolução sequidas de camadas maxPooling. Utilizo uma cada flatten para ajustar as imagens de forma que possam seguir para uma camada densa de 64 neuronios e por fim uma camada de saida com 4 neuronios, onde cada um representa a saida de um dos pontos que irão tentar identificar onde esta a roda do carro.

In [21]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(16, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='sigmoid'))

## - utilizo o calculo mse para definir o loss e uso o 'mae' como metrica, por ser um problema de regressão e quanto menor for essa metrica, mais próximo a rede esta de encontrar o valor verdadeiro a ser previsto.
## - A rede é treinada em 50 epocas, utilizando um batch_size adequado a capacidade de processamento e memoria disponivel.

In [22]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [23]:
model.fit(x_train, y_train, epochs = 50, batch_size = 12)

Epoch 1/50
48/48 [==============================] - 17s 342ms/step - loss: 0.0260 - mae: 0.1076
Epoch 2/50
48/48 [==============================] - 17s 348ms/step - loss: 0.0057 - mae: 0.0493
Epoch 3/50
48/48 [==============================] - 17s 348ms/step - loss: 0.0026 - mae: 0.0348
Epoch 4/50
48/48 [==============================] - 17s 348ms/step - loss: 0.0014 - mae: 0.0267
Epoch 5/50
48/48 [==============================] - 17s 347ms/step - loss: 8.8211e-04 - mae: 0.0212
Epoch 6/50
48/48 [==============================] - 17s 348ms/step - loss: 5.8082e-04 - mae: 0.0175
Epoch 7/50
48/48 [==============================] - 17s 351ms/step - loss: 4.2441e-04 - mae: 0.0149
Epoch 8/50
48/48 [==============================] - 17s 351ms/step - loss: 3.5288e-04 - mae: 0.0140
Epoch 9/50
48/48 [==============================] - 17s 350ms/step - loss: 3.0086e-04 - mae: 0.0131
Epoch 10/50
48/48 [==============================] - 17s 352ms/step - loss: 2.4850e-04 - mae: 0.0115
Epoch 11/50
48/

## - Realizado o teste, e apresentado o resultado, Se mantido 'len(x_teste)' como quantidade de imagens a serem apresentadas, todas as 144 serão exibidas, caso queira exibir menos, mude este valor.

In [26]:
y_pred = model.predict(x_test)

5/5 [==============================] - 1s 146ms/step


In [27]:
show_imagens(len(x_test), x_test, y_pred)